In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

import cv2
from tqdm import tqdm

In [34]:
traindf = pd.read_csv("./data/labels_training.csv",dtype=str)
def append_ext(fn):
    return fn+".tif"
traindf["id"]=traindf["id"].apply(append_ext)

In [35]:
# dir_train_images  = './data/training/'
dir_test_images   = './data/testing/'
# dir_train_labels  = './data/labels_training.csv'
# dir_test_ids      = './data/sample_submission.csv'

# labels_pd = pd.read_csv(dir_train_labels)

In [139]:
x = []
x_test = []
y = []

In [140]:
for f, tags in traindf.values:
    
    training_data_file = 'data/training/{}'.format(f)
    img = cv2.imread(training_data_file)
    x.append(cv2.resize(img, (32, 32)))
    y.append(tags)

In [141]:
y = np.array(y, np.uint8)
x = np.array(x, np.float16) / 255.

### 33% percent validation set
split = round(x.shape[0]*0.33)


In [125]:
len(y[y==1])

505

In [132]:
split = 250
x_val, x_train, y_val, y_train = x[:split], x[split:], y[:split], y[split:]

In [136]:
len(x_train)

1250

In [142]:
model = None
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32, 32, 3)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer='adam',
              metrics=['accuracy'])

## TODO: Add validation data
model.fit(x, y,
          batch_size=128,
          epochs=100,
          verbose=1)

Epoch 1/100
1500/1500 [==============================] - 4s 2ms/step - loss: 0.7085 - acc: 0.6220
Epoch 2/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.6437 - acc: 0.6633
Epoch 3/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.6392 - acc: 0.6633
Epoch 4/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.6292 - acc: 0.6640
Epoch 5/100
1500/1500 [==============================] - 2s 2ms/step - loss: 0.6194 - acc: 0.6687
Epoch 6/100
1500/1500 [==============================] - 2s 2ms/step - loss: 0.5915 - acc: 0.6687
Epoch 7/100
1500/1500 [==============================] - 2s 2ms/step - loss: 0.5811 - acc: 0.6840
Epoch 8/100
1500/1500 [==============================] - 2s 2ms/step - loss: 0.5559 - acc: 0.7000
Epoch 9/100
1500/1500 [==============================] - 2s 2ms/step - loss: 0.5356 - acc: 0.7087
Epoch 10/100
1500/1500 [==============================] - 2s 2ms/step - loss: 0.5283 - acc: 0.7280
Epoch 11/100
1500/1

1500/1500 [==============================] - 3s 2ms/step - loss: 0.0718 - acc: 0.9880
Epoch 84/100
1500/1500 [==============================] - 3s 2ms/step - loss: 0.0689 - acc: 0.9873
Epoch 85/100
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1194 - acc: 0.9613
Epoch 86/100
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1072 - acc: 0.9727
Epoch 87/100
1500/1500 [==============================] - 3s 2ms/step - loss: 0.0921 - acc: 0.9740
Epoch 88/100
1500/1500 [==============================] - 3s 2ms/step - loss: 0.0793 - acc: 0.9873
Epoch 89/100
1500/1500 [==============================] - 3s 2ms/step - loss: 0.0819 - acc: 0.9800
Epoch 90/100
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1071 - acc: 0.9680
Epoch 91/100
1500/1500 [==============================] - 3s 2ms/step - loss: 0.0820 - acc: 0.9833
Epoch 92/100
1500/1500 [==============================] - 3s 2ms/step - loss: 0.0804 - acc: 0.9813
Epoch 93/100
1500/1500 

In [149]:
training_prediction = model.predict(x, batch_size=128)

In [152]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y, training_prediction)

1.0

In [157]:
### load test data
dir_test_ids      = './data/sample_submission.csv'
labels_pd = pd.read_csv(dir_test_ids)
ids= labels_pd.id.values
x_test = []
for identifier in ids:
    img = cv2.imread("./data/testing/"+str(identifier)+'.tif')
    x_test.append(cv2.resize(img, (32, 32)))

In [159]:
x_test = np.array(x_test, np.float16) / 255.

In [160]:
test_prediction = model.predict(x_test, batch_size=128)

In [165]:
submission_file = pd.DataFrame({'id':    ids,
                                'score':  test_prediction.ravel()})
submission_file.to_csv('submission_basic_cnn.csv',
                       columns=['id','score'],
                       index=False)

In [164]:
test_prediction.ravel()

array([0.00000000e+00, 1.50975585e-03, 6.85012937e-01, 1.90600753e-03,
       1.51101977e-01, 6.91413879e-06, 7.74042010e-01, 5.59533596e-01,
       1.13898218e-02, 9.87324476e-01, 6.48200512e-05, 1.24186277e-04,
       3.39150429e-05, 9.38823521e-02, 4.79817390e-06, 1.08465552e-03,
       1.14631355e-02, 9.42898631e-01, 8.57667923e-01, 9.04835761e-02,
       5.34968376e-01, 7.86781311e-06, 8.13871622e-04, 0.00000000e+00,
       7.73826540e-02, 9.15159822e-01, 8.26561451e-03, 2.12218165e-02,
       5.87062299e-01, 4.56225872e-03, 8.51243556e-01, 8.70227814e-06,
       3.48706245e-01, 8.79582405e-01, 3.20333242e-03, 2.56007284e-01,
       9.96443212e-01, 2.08616257e-07, 9.94920790e-01, 0.00000000e+00,
       9.79549766e-01, 8.40425491e-06, 8.80633473e-01, 2.91590691e-02,
       5.25122881e-03, 1.05197847e-01, 2.96294689e-04, 1.93634629e-03,
       1.37296885e-01, 2.48014927e-03, 9.91209507e-01, 9.97096241e-01,
       3.60481650e-01, 9.97987509e-01, 8.30858946e-04, 1.19209290e-07,
      